<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [5]:
## Task 1: Mark all launch sites on a map
# Example: DataFrame with Launch Sites (replace with your df)
# Ensure df has columns: 'LaunchSite', 'Lat', 'Long'
data = {
    "LaunchSite": ["CCAFS LC-40", "CCAFS SLC-40", "KSC LC-39A", "VAFB SLC-4E"],
    "Lat": [28.5623, 28.5632, 28.5733, 34.6321],
    "Long": [-80.5774, -80.5768, -80.6469, -120.6106]
}
df = pd.DataFrame(data)

# Center the map around the mean of coordinates
map_center = [df["Lat"].mean(), df["Long"].mean()]
launch_map = folium.Map(location=map_center, zoom_start=4)

# Add markers for each launch site
for i, row in df.iterrows():
    folium.Marker(
        location=[row["Lat"], row["Long"]],
        popup=row["LaunchSite"],
        icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
    ).add_to(launch_map)

# Save map to HTML file
launch_map.save("launch_sites_map.html")

launch_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [14]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [15]:
# Task 2: Mark the success/failed launches for each site on the map
# Example data (replace with your own dataframe)
# Example data (replace with your own dataframe)
data = {
    "LaunchSite": ["CCAFS LC-40", "CCAFS SLC-40", "KSC LC-39A", "VAFB SLC-4E"],
    "Lat": [28.562302, 28.563197, 28.573255, 34.632834],
    "Long": [-80.577356, -80.576820, -80.646895, -120.610745],
    "Class": [1, 0, 1, 1]   # 1 = success, 0 = failure
}
df = pd.DataFrame(data)

# Center map
map_center = [df["Lat"].mean(), df["Long"].mean()]
launch_map = folium.Map(location=map_center, zoom_start=4)

# Add markers for each launch event
for i, row in df.iterrows():
    # Choose color based on success/failure
    color = "green" if row["Class"] == 1 else "red"
    
    folium.CircleMarker(
        location=[row["Lat"], row["Long"]],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"Site: {row['LaunchSite']} | Outcome: {'Success' if row['Class']==1 else 'Failure'}",
        tooltip=row["LaunchSite"]
    ).add_to(launch_map)

# Save and display
launch_map.save("launch_outcomes_map.html")
launch_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [19]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [16]:
import folium
from folium.plugins import MarkerCluster

# Create base map centered around Florida (roughly near CCAFS)
site_map = folium.Map(location=[28.5, -80.6], zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Example: add launch site markers to the cluster
launch_sites = {
    "CCAFS LC-40": [28.5623, -80.5774],
    "KSC LC-39A": [28.5733, -80.6469],
    "VAFB SLC-4E": [34.6321, -120.6106],
    "CCAFS SLC-40": [28.5632, -80.5768]
}

for name, coord in launch_sites.items():
    folium.Marker(
        location=coord,
        popup=name,
        icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
    ).add_to(marker_cluster)

# Show map
site_map


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [17]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red# Create a new column 'marker_color' based on the 'class' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Check first few rows
spacex_df[['Launch Site', 'class', 'marker_color']].head()


,Launch Site,class,marker_color
0,CCAFS LC-40,0,red
1,CCAFS LC-40,0,red
2,CCAFS LC-40,0,red
3,CCAFS LC-40,0,red
4,CCAFS LC-40,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
from folium import Icon

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
for index, record in spacex_df.iterrows():
    # Extract coordinates
    lat = record['Lat']
    lon = record['Long']
    site = record['Launch Site']
    outcome = "Success" if record['class'] == 1 else "Failure"
    
    # Create a folium.Marker
    marker = folium.Marker(
        location=[lat, lon],
        popup=f"{site} | {outcome}",
        icon=Icon(color='white', icon_color=record['marker_color'], icon="rocket", prefix="fa")
    )
    
    # Add to cluster
    marker_cluster.add_child(marker)

# Display map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [26]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math
import pandas as pd

def haversine(coord1, coord2):
    # Coordinates in decimal degrees (lat, lon)
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    R = 6371  # Earth radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

# Example: Launch sites
launch_sites = {
    "CCAFS LC-40": (28.5623, -80.5774),
    "KSC LC-39A": (28.5733, -80.6469),
    "VAFB SLC-4E": (34.6321, -120.6106),
}

# Example: Proximities
proximities = {
    "Orlando International Airport": (28.4312, -81.3081),
    "Cape Canaveral": (28.3922, -80.6077),
    "Vandenberg Village": (34.6922, -120.4710),
}

# Calculate distances
distance_data = []
for site, site_coord in launch_sites.items():
    for prox, prox_coord in proximities.items():
        dist_km = haversine(site_coord, prox_coord)
        distance_data.append([site, prox, round(dist_km, 2)])

df_distances = pd.DataFrame(distance_data, columns=["LaunchSite", "Proximity", "Distance_km"])
print(df_distances)


    LaunchSite                      Proximity  Distance_km
0  CCAFS LC-40  Orlando International Airport        72.88
1  CCAFS LC-40                 Cape Canaveral        19.14
2  CCAFS LC-40             Vandenberg Village      3812.94
3   KSC LC-39A  Orlando International Airport        66.51
4   KSC LC-39A                 Cape Canaveral        20.50
5   KSC LC-39A             Vandenberg Village      3806.15
6  VAFB SLC-4E  Orlando International Airport      3764.10
7  VAFB SLC-4E                 Cape Canaveral      3829.71
8  VAFB SLC-4E             Vandenberg Village        14.41


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [34]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [35]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [37]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
import math
from shapely.geometry import Point

def haversine(coord1, coord2):
    # Coordinates as (lat, lon)
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in km

    # Convert degrees to radians
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))

# Example launch site
launch_site = (28.56367, -80.57163)

# Example coastline coordinate (Cape Canaveral beach)
coastline = (28.5623, -80.5670)

# Calculate distance
distance_coastline = haversine(launch_site, coastline)
print("Distance to coastline (km):", round(distance_coastline, 2))


Distance to coastline (km): 0.48


In [38]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
import folium
from folium.features import DivIcon
import math

# --- Haversine function (no geopy needed) ---
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in km

    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

# --- Coordinates ---
launch_site = (28.56367, -80.57163)   # Example: Launch pad
coastline = (28.5623, -80.5670)       # Example: Coastline point near Cape Canaveral

# --- Calculate distance ---
distance_km = haversine(launch_site, coastline)

# --- Create folium map centered on launch site ---
m = folium.Map(location=launch_site, zoom_start=13)

# Marker for launch site
folium.Marker(
    launch_site,
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(m)

# Marker for coastline point
folium.Marker(
    coastline,
    popup="Closest Coastline",
    icon=folium.Icon(color="green", icon="flag", prefix="fa")
).add_to(m)

# Distance label at coastline point
distance_marker = folium.Marker(
    location=coastline,
    icon=DivIcon(
        icon_size=(150,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(m)

# Draw a line between launch site and coastline
folium.PolyLine([launch_site, coastline], color="red", weight=2, opacity=0.8).add_to(m)

# Save map
m.save("launch_to_coastline.html")
m


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [40]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
import folium
from folium.features import DivIcon
import math

# --- Haversine distance ---
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in km

    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

# --- Coordinates ---
launch_site = (28.56367, -80.57163)   # Launch site
coastline = (28.5623, -80.5670)       # Closest coastline (example)

# --- Calculate distance ---
distance_km = haversine(launch_site, coastline)

# --- Create folium map ---
site_map = folium.Map(location=launch_site, zoom_start=14)

# Add markers
folium.Marker(
    launch_site,
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(site_map)

folium.Marker(
    coastline,
    popup="Closest Coastline",
    icon=folium.Icon(color="green", icon="flag", prefix="fa")
).add_to(site_map)

# Distance label
folium.Marker(
    location=coastline,
    icon=DivIcon(
        icon_size=(150,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(site_map)

# --- Draw polyline between launch site and coastline ---
coordinates = [launch_site, coastline]
lines = folium.PolyLine(locations=coordinates, weight=2, color="red")
site_map.add_child(lines)

# Save map
site_map.save("launch_to_coastline_line.html")
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [41]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [43]:
import folium
from folium.features import DivIcon
import math

# --- Haversine distance ---
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in km

    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

# --- Example Launch Site ---
launch_site = (28.56367, -80.57163)  # Cape Canaveral LC-40

# --- Proximities dictionary ---
proximities = {
    "Closest City (Cape Canaveral)": (28.3922, -80.6077),
    "Closest Airport (Orlando Int’l)": (28.4312, -81.3081),
    "Closest Highway": (28.5100, -80.7500),
    "Closest Railway": (28.4100, -80.6000)
}

# --- Create map ---
site_map = folium.Map(location=launch_site, zoom_start=9)

# Add launch site marker
folium.Marker(
    launch_site,
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(site_map)

# --- Add proximity markers with distance ---
for name, coord in proximities.items():
    distance_km = haversine(launch_site, coord)

    # Add marker
    folium.Marker(
        coord,
        popup=f"{name}: {distance_km:.2f} km",
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(site_map)

    # Add distance label
    folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
        )
    ).add_to(site_map)

    # Draw line from launch site to proximity
    folium.PolyLine([launch_site, coord], color="red", weight=2, opacity=0.7).add_to(site_map)

# Save map
site_map.save("launch_proximities_map.html")
site_map

In [44]:
import folium
from folium.features import DivIcon
import math

# --- Haversine distance ---
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth radius in km

    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

# --- Launch Site ---
launch_site = (28.56367, -80.57163)  # Cape Canaveral LC-40

# --- Proximities ---
proximities = {
    "Closest City (Cape Canaveral)": (28.3922, -80.6077),
    "Closest Airport (Orlando Int’l)": (28.4312, -81.3081),
    "Closest Highway": (28.5100, -80.7500),
    "Closest Railway": (28.4100, -80.6000)
}

# --- Create map ---
site_map = folium.Map(location=launch_site, zoom_start=9)

# Launch site marker
folium.Marker(
    launch_site,
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(site_map)

# Proximity markers + lines
for name, coord in proximities.items():
    distance_km = haversine(launch_site, coord)

    # Proximity marker
    folium.Marker(
        coord,
        popup=f"{name}: {distance_km:.2f} km",
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(site_map)

    # Distance label
    folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
        )
    ).add_to(site_map)

    # --- Draw line from launch site to proximity ---
    folium.PolyLine(
        locations=[launch_site, coord],
        color="red",
        weight=2,
        opacity=0.7
    ).add_to(site_map)

# Save map
site_map.save("launch_proximities_with_lines.html")
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
